# Optimizing QAOA

by Pantita Palittapongarnpim

Now that we have the classes for QAOA circuits, we need to optimize the circuits.

### Package Versions

QuTiP: 4.5.2

Python: 3.8.6

### Loading Packages

In [1]:
import numpy as np
import qutip as qt

### QAOA Classes

In [2]:
class Graph:
    def __init__(self, total_qubit, J):
        self.total_qubit=total_qubit #integer
        self.J=J #matrix: undirected (symmetric)

In [3]:
class QAOA:
    def __init__(self, gamma, beta, p, graph):
        ## what does graph has to contain:
        # total_qubit -- integer
        # J -- double matrix        
        self.gamma=gamma
        self.beta=beta
        self.p=p
        self.J=graph.J
        self.total_qubit=graph.total_qubit
        
    def Uz(self,i,j,l):
        opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
        Hvar=-1j*gamma[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def Ux(self,i,l):
        opr=qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(self.total_qubit)])
        Hvar=-1j*beta[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def circuit(self):
        Had=qt.tensor([qt.qip.operations.snot() for k in range(self.total_qubit)])
        qaoa_circ=Had
        for l in range(self.p):
            #operate the Uz gates
            for i in range(self.total_qubit):
                for j in range(i,self.total_qubit):
                    if i!= j:
                        qaoa_circ=self.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(self.total_qubit):
                qaoa_circ=self.Ux(i,l)*qaoa_circ
        return(qaoa_circ)
    
    def state(self):
        gnd=qt.basis(2,0)
        state=qt.tensor([gnd for i in range(self.total_qubit)])
        state=self.circuit()*state
        return(state)
 

### Optimization